# ECE449: Individaul Project
Jie Wang

Nov. 11, 2023

## Project Description

In this project, you need to solve an image classification task using coil-20-proc dataset.

This dataset consists of 1,440 grayscale images of 20 objects (72 images per object). 
- Half of each category is for training and half for testing. (e.g. 36 images for training and another 36 images for testing).

Through this project, you can learn how to customize dataset loading, design deep models, and train and test models. 

This is an individual project, so avoid copying others' code. Additionally, you need to print the accuracy results during the training process. 

> You should use a Python Jupyter Notebook to write down the process and accuracy results.


## The COIL-20 (Columbia Object Image Library) dataset
a collection of grayscale images of 20 different objects. 

- The "proc" in "COIL-20-proc" indicates that the images in this dataset have been preprocessed. photographed on a motorized turntable against a black background to provide a 360-degree view. The objects are varied, including toys, household items, and office supplies, to provide a range of shapes and complexities.


The preprocessing typically involves normalizing the images in terms of scale and orientation, and the background is often removed to isolate the object of interest. This makes the dataset particularly useful for computer vision tasks like object recognition and classification, where consistency in the input data can significantly improve the performance of machine learning models.


# 1. Complete the custom dataset, and dataloader.
Fill in the code to complete the custom dataset and dataloader, you can refer to https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [12]:
import os
import random
import glob

import argparse
import logging

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

from matplotlib import pyplot as plt
from tqdm import tqdm

class COIL20Dataset(Dataset):
    def __init__(self, image_list, transform=None):
        self.image_list = image_list
        self.transform = transform

    def __getitem__(self, index):
        path = self.image_list[index]
        # print(path)
        image= Image.open(path)
        # Extracting label from file names
        label = int(os.path.basename(os.path.dirname(path))) -1 
        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.image_list)

# the COIL-20 images are stored in a directory as below:
# - coil-20-proc/
#       - 01/obj1__0.png, obj1__1.png, ...
#       - 02/...
#       - ...
#       - 20/....
root = 'coil-20-proc'

ddd = COIL20Dataset(root)


In [13]:
def read_image_pair(root_dir):
    image_list = []
    
    # Iterate over all directories in the root_dir
    for label_dir in os.listdir(root_dir):
        label_path = os.path.join(root_dir, label_dir)
        # print(label_path)
        if os.path.isdir(label_path):
            # Extract label from the directory name
            label = int(label_dir)  # start from 1 to 20
            # Add each image to the list
            for image_file in glob.glob(os.path.join(label_path, '*.png')):
                image_list.append(image_file)
    return image_list



In [14]:
import numpy as np

def load_coil20_dataset(base_directory='coil-20-proc', transform=None, test_split=0.2):
    # Load the full dataset list
    dataset_list = read_image_pair(base_directory)

    # Splitting the dataset into train and test sets
    dataset_size = len(dataset_list)
    indices = list(range(dataset_size))

    split = int(np.floor(test_split * dataset_size))
    random.shuffle(indices)
    train_indices, test_indices = indices[split:], indices[:split]

    # Split dataset into training and test sets
    train_image_list = [dataset_list[i] for i in train_indices]
    test_image_list = [dataset_list[i] for i in test_indices]
    print(len(train_image_list))
    # Creating datasets
    train_dataset = COIL20Dataset(train_image_list, transform=transform)
    test_dataset = COIL20Dataset(test_image_list, transform=transform)

    # Creating data loaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, test_loader
train_loader, test_loader = load_coil20_dataset()

1152


In [15]:
print(train_loader.__len__() ) 
print(test_loader.__len__() ) 

36
9


In [16]:
import matplotlib.pyplot as plt
import torchvision

def show_images_from_dataloader(dataloader):
    # Get a batch of training data
    images, labels = next(iter(dataloader))

    # Make a grid from batch
    out = torchvision.utils.make_grid(images)

    plt.figure(figsize=(15, 15))
    plt.imshow(out.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')
    plt.axis('off')
    plt.show()

    # Print labels
    print('Labels:', labels.numpy())

# Example usage with your train_loader
# show_images_from_dataloader(train_loader)


In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# 2. Implementing a Neural Network
Fill in the code to complete the custom model, you can refer to https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [18]:
class Classify_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128*128, 512),  
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 20)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


# 3. Customize some arguments
Usually, this function is not used for jupyter notebook, as it violates the nature of interacive design.

Still, I set it mannually for the argument input. 

# 4. Train and Test the model.
Fill in the code to complete the training and testing, and print the results and losses. You can refer to https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html and https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [19]:
class Coil20_model:
    def __init__(self, train_loader, test_loader):
        self.train_loader = train_loader 
        self.test_loader = test_loader
        self.Mymodel = Classify_NN()
        self.criterion = nn.CrossEntropyLoss()  
        self.optimizer = torch.optim.Adam(self.Mymodel.parameters(), lr=0.001) 

    def _train(self, dataloader):
        train_loss = 0
        n_correct = 0
        n_samples = 0
        self.Mymodel.train()
        for images, labels in dataloader:
            images, labels = images.float(), labels.long()
            outputs = self.Mymodel(images)
            loss = self.criterion(outputs, labels)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        avg_loss = train_loss / len(dataloader)
        accuracy = n_correct / n_samples
        return avg_loss, accuracy

    def _test(self, dataloader):
        test_loss = 0
        n_correct = 0
        n_samples = 0
        self.Mymodel.eval()
        with torch.no_grad():
            for images, labels in dataloader:
                images, labels = images.float(), labels.long()
                outputs = self.Mymodel(images)
                loss = self.criterion(outputs, labels)

                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                n_samples += labels.size(0)
                n_correct += (predicted == labels).sum().item()

        avg_loss = test_loss / len(dataloader)
        accuracy = n_correct / n_samples
        return avg_loss, accuracy

    def run(self, num_epochs):
        for epoch in range(num_epochs):
            train_loss, train_accuracy = self._train(self.train_loader)
            print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}')
            print("start testing")
            test_loss, test_accuracy = self._test(self.test_loader)
            print(f'Epoch {epoch+1}: Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}')


In [20]:
    # def _visualize(self, accuracy, val_accuracy):
    #     plt.plot(accuracy, label='accuracy')
    #     plt.plot(val_accuracy, label='val_accuracy')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Accuracy')
    #     plt.ylim([0, 1])
    #     plt.legend(loc='lower right')
    #     plt.show()


In [22]:
transform = transforms.Compose([
 transforms.Resize((128, 128)),
 transforms.ToTensor(),
 ])
train,test = load_coil20_dataset('coil-20-proc/',transform)
ece449_model = Coil20_model(train,test )  
ece449_model.run(5)  




1152
Epoch 1: Train Loss: 1.5518, Accuracy: 0.54
Epoch 1: Test Loss: 0.6255, Accuracy: 0.83
Epoch 2: Train Loss: 0.3411, Accuracy: 0.90
Epoch 2: Test Loss: 0.2276, Accuracy: 0.95
Epoch 3: Train Loss: 0.1035, Accuracy: 0.98
Epoch 3: Test Loss: 0.0722, Accuracy: 0.98
Epoch 4: Train Loss: 0.0386, Accuracy: 1.00
Epoch 4: Test Loss: 0.0634, Accuracy: 0.99
Epoch 5: Train Loss: 0.0534, Accuracy: 0.99
Epoch 5: Test Loss: 0.0411, Accuracy: 0.99


In [25]:
image_path = 'coil-20-proc/04/obj4__66.png'
def evaluate_single_image(model, image):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        return predicted.item()

image = Image.open(image_path)

predicted_label = evaluate_single_image(ece449_model.Mymodel, image)
print(f"Predicted Label: {predicted_label}")


AttributeError: flatten

In [ ]:
# To save the model
torch.save(ece449_model.Mymodel.state_dict(), 'coil20_model.pth')

# To load the model
ece449_model.Mymodel.load_state_dict(torch.load('coil20_model.pth'))